In [1]:
import os

In [2]:
%pwd

'c:\\Users\\adity\\Desktop\\lungs classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\adity\\Desktop\\lungs classification'

In [6]:
pip install keras-tuner


Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import time
from pathlib import Path
from dataclasses import dataclass
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.Classifier.utils.common import read_yaml, create_directories
from src.Classifier.constants import *
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s: %(levelname)s]: %(message)s"
)
logger = logging.getLogger(__name__)

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_classes: int
    params_learning_rate: float

class CustomModelCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, filepath, monitor='val_accuracy', save_best_only=True, mode='max'):
        super().__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.save_best_only = save_best_only
        self.mode = mode
        self.best_value = -float('inf') if mode == 'max' else float('inf')

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            return
        if (self.mode == 'max' and current > self.best_value) or \
           (self.mode == 'min' and current < self.best_value):
            logger.info(f"Saving best model to {self.filepath}")
            try:
                self.model.save(self.filepath, save_format='h5', include_optimizer=True)
            except Exception as e:
                logger.warning(f"Failed to save full model, saving weights instead due to: {e}")
                self.model.save_weights(self.filepath.replace('.h5', '.weights.h5'))
            self.best_value = current

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.total_epochs = 0
        self.model = None
        self.train_generator = None
        self.valid_generator = None
        self.steps_per_epoch = None
        self.validation_steps = None
        logger.info("Training pipeline initialized")

    def get_base_model(self):
        logger.info(f"Loading base model from {self.config.updated_base_model_path}")
        self.model = load_model(str(self.config.updated_base_model_path))
        for layer in self.model.layers:
            layer.trainable = True
        logger.info("Base model loaded and all layers set as trainable")

    def _get_data_generators(self):
        logger.info("Preparing data generators...")
        if self.config.params_is_augmentation:
            train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest')
        else:
            train_datagen = ImageDataGenerator(rescale=1./255)

        test_datagen = ImageDataGenerator(rescale=1./255)

        self.train_generator = train_datagen.flow_from_directory(
            directory=str(self.config.training_data),
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            shuffle=True)

        self.valid_generator = test_datagen.flow_from_directory(
            directory=str(self.config.validation_data),
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            shuffle=False)

        self.steps_per_epoch = self.train_generator.samples // self.config.params_batch_size
        self.validation_steps = self.valid_generator.samples // self.config.params_batch_size

    def _compile_model(self, learning_rate):
        self.model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy'])
        logger.info(f"Model compiled with learning rate: {learning_rate}")

    def train(self):
        logger.info("Starting training...")
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        checkpoint_path = str(Path(self.config.root_dir) / f"model_checkpoint_{timestamp}.h5")
        tb_log_dir = str(Path(self.config.root_dir) / "logs" / timestamp)

        callbacks = [
            CustomModelCheckpoint(
                filepath=checkpoint_path,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'),
            tf.keras.callbacks.TensorBoard(
                log_dir=tb_log_dir,
                histogram_freq=1)
        ]

        self._compile_model(self.config.params_learning_rate)
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callbacks)

        self.total_epochs += self.config.params_epochs
        logger.info(f"Training complete. Total epochs: {self.total_epochs}")

    def get_trained_model(self):
        return self.model

    def save_model(self):
        """Save the trained model in both Keras (.h5) and TFLite formats."""
        base_path = str(self.config.trained_model_path.with_suffix(''))
        h5_path = base_path + ".h5"
        tflite_path = base_path + ".tflite"

        # Ensure the directory exists
        create_directories([Path(h5_path).parent])

        # Save Keras model
        try:
            self.model.save(h5_path, save_format='h5', include_optimizer=True)
            logger.info(f"Keras model saved to {h5_path}")
        except Exception as e:
            logger.warning(f"Failed to save Keras model due to: {e}")
            self.model.save_weights(h5_path.replace('.h5', '.weights.h5'))
            logger.info(f"Model weights saved to {h5_path.replace('.h5', '.weights.h5')}")

        # Save TFLite model
        try:
            converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
            converter.optimizations = [tf.lite.Optimize.DEFAULT]
            tflite_model = converter.convert()
            with open(tflite_path, "wb") as f:
                f.write(tflite_model)
            logger.info(f"Compressed TFLite model saved to {tflite_path}")
        except Exception as e:
            logger.error(f"Failed to save TFLite model due to: {e}")

if __name__ == "__main__":
    try:
        config = read_yaml(CONFIG_FILE_PATH)
        params = read_yaml(PARAMS_FILE_PATH)

        training_config = TrainingConfig(
            root_dir=Path(config['training']['root_dir']),
            trained_model_path=Path(config['training']['trained_model_path']),
            updated_base_model_path=Path(config['prepare_base_model']['updated_base_model_path']),
            training_data=Path(config['data_ingestion']['unzip_dir']) / "The IQ-OTHNCCD lung cancer dataset" / "Train",
            validation_data=Path(config['data_ingestion']['unzip_dir']) / "The IQ-OTHNCCD lung cancer dataset" / "Test",
            params_epochs=params['EPOCHS'],
            params_batch_size=params['BATCH_SIZE'],
            params_is_augmentation=params['AUGMENTATION'],
            params_image_size=params['IMAGE_SIZE'],
            params_classes=params['CLASSES'],
            params_learning_rate=params['LEARNING_RATE']
        )

        if not training_config.training_data.exists():
            raise FileNotFoundError(f"Training data not found: {training_config.training_data}")
        if not training_config.validation_data.exists():
            raise FileNotFoundError(f"Validation data not found: {training_config.validation_data}")

        trainer = Training(training_config)
        trainer.get_base_model()
        trainer._get_data_generators()
        trainer.train()
        trainer.save_model()

        logger.info("Training and saving pipeline completed successfully")

    except Exception as e:
        logger.exception(f"Error in training pipeline: {e}")
        raise e

[2025-05-11 10:49:16,068: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-11 10:49:16,079: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-11 10:49:16,083: INFO: 4025821603: Training pipeline initialized]
[2025-05-11 10:49:16,087: INFO: 4025821603: Loading base model from artifacts\prepare_base_model\base_model_updated.h5]
[2025-05-11 10:49:17,804: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2025-05-11 10:49:17,817: INFO: 4025821603: Base model loaded and all layers set as trainable]
[2025-05-11 10:49:17,820: INFO: 4025821603: Preparing data generators...]
Found 864 images belonging to 3 classes.
Found 230 images belonging to 3 classes.
[2025-05-11 10:49:17,994: INFO: 4025821603: Starting training...]
[2025-05-11 10:49:18,016: INFO: 4025821603: Model compiled with learning rate: 0.001]
Epoch 1/15
14/14 ━━━━━━━━━

In [9]:
def save_keras_and_tflite(model, save_path):
    save_path = str(save_path)

    # Ensure .h5 path
    if not save_path.endswith(".h5"):
        save_path += ".h5"

    # Save the full Keras model in HDF5 format
    model.save(save_path)
    logger.info(f"Keras model saved to {save_path}")

    # Prepare TFLite path
    tflite_path = save_path.replace(".h5", ".tflite")

    # Convert to TFLite with dynamic range quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    # Save the compressed TFLite model
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)

    logger.info(f"Compressed TFLite model saved to {tflite_path}")
save_keras_and_tflite(trainer.get_trained_model(), trainer.config.trained_model_path)


[2025-05-11 10:20:02,903: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_5896\2347216890.py", line 25, in <module>
    save_keras_and_tflite(trainer.get_trained_model(), trainer.config.trained_model_path)
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_5896\2347216890.py", line 9, in save_keras_and_tflite
    model.save(save_path)
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\adity\anaconda3\Lib\copy.py", line 136, in deepcopy
    y = copier(x, memo)
        ^^^^^^^^^^^^^^^
  File "C:\Users\adity\anaconda3\Lib\copy.py", line 221, in _deepcopy_dict
    y[deepcopy(key, memo)] = deepcopy(value, memo)
                            

In [9]:
import os
import time
from pathlib import Path
from dataclasses import dataclass
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.Classifier.utils.common import read_yaml, create_directories
from src.Classifier.constants import *
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s: %(levelname)s]: %(message)s"
)
logger = logging.getLogger(__name__)

# Configuration dataclass
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_classes: int
    params_learning_rate: float

# Custom callback to save best model
class CustomModelCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, filepath, monitor='val_accuracy', save_best_only=True, mode='max'):
        super().__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.save_best_only = save_best_only
        self.mode = mode
        self.best_value = -float('inf') if mode == 'max' else float('inf')

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            return
        if (self.mode == 'max' and current > self.best_value) or \
           (self.mode == 'min' and current < self.best_value):
            logger.info(f"Saving model to {self.filepath}")
            self.model.save(self.filepath)  # Automatically saves in new `.keras` format
            self.best_value = current

# Training pipeline
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.total_epochs = 0
        logger.info("Training pipeline initialized")

    def get_base_model(self):
        logger.info(f"Loading base model from {self.config.updated_base_model_path}")
        self.model = load_model(str(self.config.updated_base_model_path))

        # Make all layers trainable
        for layer in self.model.layers:
            layer.trainable = True

        logger.info("Base model loaded and set as trainable")

    def _get_data_generators(self):
        logger.info("Creating data generators...")

        # Training generator
        if self.config.params_is_augmentation:
            train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest'
            )
        else:
            train_datagen = ImageDataGenerator(rescale=1./255)

        test_datagen = ImageDataGenerator(rescale=1./255)

        self.train_generator = train_datagen.flow_from_directory(
            directory=str(self.config.training_data),
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            shuffle=True
        )

        self.valid_generator = test_datagen.flow_from_directory(
            directory=str(self.config.validation_data),
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            shuffle=False
        )

        logger.info(f"Class indices: {self.train_generator.class_indices}")
        self.steps_per_epoch = self.train_generator.samples // self.config.params_batch_size
        self.validation_steps = self.valid_generator.samples // self.config.params_batch_size

    def _compile_model(self, learning_rate=0.001):
        self.model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        self.model.summary()
        logger.info(f"Model compiled with learning rate: {learning_rate}")

    def train(self):
        logger.info("Starting training...")

        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        checkpoint_path = str(Path(self.config.root_dir) / f"model_checkpoint_{timestamp}.keras")
        tb_log_dir = str(Path(self.config.root_dir) / "logs" / timestamp)

        callbacks = [
            CustomModelCheckpoint(
                filepath=checkpoint_path,
                monitor='val_accuracy',
                save_best_only=True,
                mode='max'
            ),
            tf.keras.callbacks.TensorBoard(
                log_dir=tb_log_dir,
                histogram_freq=1
            )
        ]

        # Initial training
        history = self.model.fit(
            self.train_generator,
            epochs=15,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callbacks
        )
        self.total_epochs += 15

        # Fine-tuning with lower learning rate
        self._compile_model(learning_rate=self.config.params_learning_rate / 10)

        history = self.model.fit(
            self.train_generator,
            epochs=15,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callbacks
        )
        self.total_epochs += 15

        logger.info(f"Training completed. Total epochs: {self.total_epochs}")
        return history

def save_model(self):

    save_path = str(self.config.trained_model_path)

    # Ensure .h5 path
    if not save_path.endswith(".h5"):
        save_path += ".h5"

    # Save the full Keras model in HDF5 format
    self.model.save(save_path)
    logger.info(f"Keras model saved to {save_path}")

    # Prepare TFLite path
    tflite_path = save_path.replace(".h5", ".tflite")

    # Convert to TFLite with dynamic range quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(self.model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    # Save the compressed TFLite model
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)

    logger.info(f"Compressed TFLite model saved to {tflite_path}")

     


# Entry point
if __name__ == "__main__":
    try:
        config = read_yaml(CONFIG_FILE_PATH)
        params = read_yaml(PARAMS_FILE_PATH)

        training_config = TrainingConfig(
            root_dir=Path(config['training']['root_dir']),
            trained_model_path=Path(config['training']['trained_model_path']),
            updated_base_model_path=Path(config['prepare_base_model']['updated_base_model_path']),
            training_data=Path(config['data_ingestion']['unzip_dir']) / "The IQ-OTHNCCD lung cancer dataset" / "Train",
            validation_data=Path(config['data_ingestion']['unzip_dir']) / "The IQ-OTHNCCD lung cancer dataset" / "Test",
            params_epochs=params['EPOCHS'],
            params_batch_size=params['BATCH_SIZE'],
            params_is_augmentation=params['AUGMENTATION'],
            params_image_size=params['IMAGE_SIZE'],
            params_classes=params['CLASSES'],
            params_learning_rate=params['LEARNING_RATE']
        )

        # Sanity checks
        if not training_config.training_data.exists():
            raise FileNotFoundError(f"Training data path not found: {training_config.training_data}")
        if not training_config.validation_data.exists():
            raise FileNotFoundError(f"Validation data path not found: {training_config.validation_data}")

        trainer = Training(training_config)
        trainer.get_base_model()
        trainer._get_data_generators()
        trainer._compile_model()
        trainer.train()
        trainer.save_model()

        logger.info("Training pipeline completed successfully")

    except Exception as e:
        logger.exception(f"Error in training pipeline: {e}")
        raise e


[2025-05-10 23:16:03,388: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-10 23:16:03,400: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-10 23:16:03,408: INFO: 2007048935: Training pipeline initialized]
[2025-05-10 23:16:03,410: INFO: 2007048935: Loading base model from artifacts\prepare_base_model\base_model_updated.h5]
[2025-05-10 23:16:05,040: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2025-05-10 23:16:05,054: INFO: 2007048935: Base model loaded and set as trainable]
[2025-05-10 23:16:05,055: INFO: 2007048935: Creating data generators...]
Found 864 images belonging to 3 classes.
Found 230 images belonging to 3 classes.
[2025-05-10 23:16:05,172: INFO: 2007048935: Class indices: {'Bengin cases': 0, 'Malignant cases': 1, 'Normal cases': 2}]


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,213,926 (16.07 MB)

 Trainable params: 4,171,903 (15.91 MB)

...

 Total params: 4,213,926 (16.07 MB)

 Trainable params: 4,171,903 (15.91 MB)

 Non-trainable params: 42,023 (164.16 KB)

[2025-05-10 23:35:50,013: INFO: 2007048935: Model compiled with learning rate: 0.0001]
Epoch 1/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 213s 9s/step - accuracy: 0.9277 - loss: 0.1898 - val_accuracy: 0.2556 - val_loss: 1.8773
Epoch 2/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 808ms/step - accuracy: 0.9167 - loss: 0.2257 - val_accuracy: 0.2500 - val_loss: 1.8990
Epoch 3/15
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9317 - loss: 0.1561[2025-05-10 23:41:31,782: INFO: 2007048935: Saving model to artifacts\training\model_checkpoint_2025-05-10-23-16-05.keras]
14/14 ━━━━━━━━━━━━━━━━━━━━ 115s 8s/step - accuracy: 0.9317 - loss: 0.1566 - val_accuracy: 0.2722 - val_loss: 2.0587
Epoch 4/15
 1/14 ━━━━━━━━━━━━━━━━━━━━ 1:35 7s/step - accuracy: 0.9833 - loss: 0.0865[2025-05-10 23:41:52,266: INFO: 2007048935: Saving model to artifacts\training\model_checkpoint_2025-05-10-23-16-05.keras]
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 986ms/step - accuracy: 0.9833 - loss: 0.0865 - val_accuracy: 0.2778 - val_loss: 2.0532
Epoch 5/15


Traceback (most recent call last):
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_18712\2007048935.py", line 234, in <module>
    raise e
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_18712\2007048935.py", line 228, in <module>
    trainer.save_model()
    ^^^^^^^^^^^^^^^^^^
AttributeError: 'Training' object has no attribute 'save_model'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\IPython\core\interactiveshell.py", line 2173, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\IPython\core\ultratb.py", line 1182, in

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [11]:
from src.Classifier.constants import *
from src.Classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [10]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-04-05 17:56:02,624: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-05 17:56:02,627: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-05 17:56:02,629: INFO: common: created directory at: artifacts]
[2025-04-05 17:56:02,632: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-04-05 17:56:02,634: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-04-05 17:56:02,637: INFO: common: created directory at: artifacts\training]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "box\\box.py", line 594, in box.box.Box.__getitem__
KeyError: 'EPOCHS'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "box\\box.py", line 633, in box.box.Box.__getattr__
  File "box\\box.py", line 621, in box.box.Box.__getitem__
box.exceptions.BoxKeyError: "'EPOCHS'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "box\\box.py", line 635, in box.box.Box.__getattr__
AttributeError: 'ConfigBox' object has no attribute 'EPOCHS'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "box\\config_box.py", line 29, in box.config_box.ConfigBox.__getattr__
  File "box\\box.py", line 649, in box.box.Box.__getattr__
box.exceptions.BoxKeyError: "'ConfigBox' object has no attribute 'EPOCHS'"

During handling of the above exception, another exception occurred:

Traceback (

[2025-04-11 01:11:20,423: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-11 01:11:20,429: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-11 01:11:20,431: INFO: common: created directory at: artifacts]
[2025-04-11 01:11:20,433: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-04-11 01:11:20,434: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-04-11 01:11:20,436: INFO: common: created directory at: artifacts\training]


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,213,926 (16.07 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_19632\1080969578.py", line 192, in <module>
    raise e
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_19632\1080969578.py", line 188, in <module>
    training.train_valid_generator()
  File "C:\Users\adity\AppData\Local\Temp\ipykernel_19632\1080969578.py", line 130, in train_valid_generator
    self.valid_generator = valid_datagenerator.flow_from_directory(
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\keras\src\legacy\preprocessing\image.py", line 1138, in flow_from_directory
    return DirectoryIterator(
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\ker

In [6]:
import sys
import os


import os
import time
from pathlib import Path
from dataclasses import dataclass
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from src.Classifier.utils.common import read_yaml, create_directories
from src.Classifier.constants import *


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_classes: int


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        return PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "split_dataset", "Train")


        create_directories([Path(training.root_dir)])

        return TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_classes=params.CLASSES
        )


class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        base_model = EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=tuple(self.config.params_image_size)
        )
        base_model.trainable = False

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.5)(x)
        output = Dense(self.config.params_classes, activation='softmax')(x)

        self.model = Model(inputs=base_model.input, outputs=output)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        self.model.summary()

    def train_valid_generator(self):
        datagenerator_kwargs = dict(rescale=1. / 255, validation_split=0.2)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


if __name__ == "__main__":
    try:
        config = ConfigurationManager()

        prepare_callbacks_config = config.get_prepare_callback_config()
        prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
        callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

        training_config = config.get_training_config()
        training = Training(config=training_config)
        training.get_base_model()
        training.train_valid_generator()
        training.train(callback_list=callback_list)

    except Exception as e:
        raise e


[2025-04-06 00:11:28,170: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-06 00:11:28,173: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-06 00:11:28,174: INFO: common: created directory at: artifacts]
[2025-04-06 00:11:28,175: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-04-06 00:11:28,176: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-04-06 00:11:28,177: INFO: common: created directory at: artifacts\training]


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,213,926 (16.07 MB)

 Trainable params: 164,355 (642.01 KB)

...

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4583 - loss: 1.0408

c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


[2025-04-06 00:12:31,322: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
21/21 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.4583 - loss: 1.0400 - val_accuracy: 0.3250 - val_loss: 0.9934
Epoch 2/20
 1/21 ━━━━━━━━━━━━━━━━━━━━ 9s 489ms/step - accuracy: 0.4062 - loss: 1.0621

c:\Users\adity\Desktop\lungs classification\myenv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


[2025-04-06 00:12:35,506: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 177ms/step - accuracy: 0.4062 - loss: 1.0621 - val_accuracy: 0.5562 - val_loss: 0.9778
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4760 - loss: 0.9776[2025-04-06 00:13:00,376: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.4760 - loss: 0.9777 - val_accuracy: 0.4750 - val_loss: 0.9764
Epoch 4/20
 1/21 ━━━━━━━━━━━

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image


# === CONFIGURATION ===
MODEL_PATH = "C:/Users/adity/Desktop/lungs classification/artifacts/training/model.h5"  # Adjust if needed
IMAGE_PATH = "C:/Users/adity/Desktop/lungs classification/artifacts/data_ingestion/split_dataset/test/Normal cases/Normal case (4).jpg"       # 🔁 Replace with your image path
IMAGE_SIZE = (224, 224)                     # 🔁 Match with your params.IMAGE_SIZE
CLASS_NAMES = ['Bengin','malignant', 'normal']          # 🔁 Replace with actual class folder names


# === LOAD MODEL ===
model = tf.keras.models.load_model(MODEL_PATH)

# === PREPROCESS IMAGE ===
img = image.load_img(IMAGE_PATH, target_size=IMAGE_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Same rescaling used during training

# === PREDICT ===
predictions = model.predict(img_array)
predicted_index = np.argmax(predictions[0])
predicted_label = CLASS_NAMES[predicted_index]
confidence = np.max(predictions[0]) * 100

# === OUTPUT ===
print(f"Predicted class: {predicted_label} ({confidence:.2f}%)")

# === OPTIONAL: Show the image ===



[2025-04-06 00:19:38,878: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted class: malignant (50.48%)
